In [ ]:
import numpy as np

Hodgkin Huxley Model

---



In [ ]:
class HodgkinHuxleyModel:
    def __init__(self, V0=-65.0, n0=0.32, m0=0.05, h0=0.6, C=1.0, g_Na=120.0, g_K=36.0, g_L=0.3,
                 E_Na=50.0, E_K=-77.0, E_L=-54.4):

        self.V0 = V0
        self.n0 = n0
        self.m0 = m0
        self.h0 = h0
        self.C = C
        self.g_Na = g_Na
        self.g_K = g_K
        self.g_L = g_L
        self.E_Na = E_Na
        self.E_K = E_K
        self.E_L = E_L

    @staticmethod
    def alpha_n(V):

        return (0.01 * (V + 55.0)) / (1.0 - np.exp(-(V + 55.0) / 10.0))

    @staticmethod
    def beta_n(V):

        return 0.125 * np.exp(-(V + 65.0) / 80.0)

    @staticmethod
    def alpha_m(V):

        return (0.1 * (V + 40.0)) / (1.0 - np.exp(-(V + 40.0) / 10.0))

    @staticmethod
    def beta_m(V):

        return 4.0 * np.exp(-(V + 65.0) / 18.0)

    @staticmethod
    def alpha_h(V):

        return 0.07 * np.exp(-(V + 65.0) / 20.0)

    @staticmethod
    def beta_h(V):

        return 1.0 / (1.0 + np.exp(-(V + 35.0) / 10.0))

    def get_sodium_current(self, V, m, h):

        I_Na = self.g_Na * (m**3) * h * (V - self.E_Na)
        return I_Na

    def get_potassium_current(self, V, n):

        I_K = self.g_K * (n**4) * (V - self.E_K)
        return I_K

    def get_leak_current(self, V):

        I_L = self.g_L * (V - self.E_L)
        return I_L

    def get_currents(self, V, n, m, h):

        I_Na = self.get_sodium_current(V, m, h)
        I_K = self.get_potassium_current(V, n)
        I_L = self.get_leak_current(V)

        return I_Na, I_K, I_L

    def get_initial_state(self):

        return [self.V0, self.n0, self.m0, self.h0]

    def set_initial_state(self, V0=-65.0, n0=0.32, m0=0.05, h0=0.6):

        self.V0 = V0
        self.n0 = n0
        self.m0 = m0
        self.h0 = h0

    def __call__(self, Y, I=0):

        V, n, m, h = Y

        I_Na = self.get_sodium_current(V, m, h)
        I_K = self.get_potassium_current(V, n)
        I_L = self.get_leak_current(V)
        dVdt = (I - I_Na - I_K - I_L) / self.C
        dndt = self.alpha_n(V) * (1.0 - n) - self.beta_n(V) * n
        dmdt = self.alpha_m(V) * (1.0 - m) - self.beta_m(V) * m
        dhdt = self.alpha_h(V) * (1.0 - h) - self.beta_h(V) * h

        return [dVdt, dndt, dmdt, dhdt]


MorrisLecar Model

---



In [ ]:
class MorrisLecarModel:
    def __init__(self, V0=-60.9, w0=0.015, C=1, g_Ca=4, g_K=8, g_L=2, V_Ca=120, V_K=-84, V_L=-60,
                 phi=0.04, V_1=-1.2, V_2=18, V_3=2, V_4=30, V_5=-40, V_6=10, V_7=0.4, V_8=18):

        self.V0 = V0
        self.w0 = w0
        self.C = C
        self.g_Ca = g_Ca
        self.g_K = g_K
        self.g_L = g_L
        self.V_Ca = V_Ca
        self.V_K = V_K
        self.V_L = V_L
        self.phi = phi
        self.V_1 = V_1
        self.V_2 = V_2
        self.V_3 = V_3
        self.V_4 = V_4
        self.V_5 = V_5
        self.V_6 = V_6
        self.V_7 = V_7
        self.V_8 = V_8

    def m_inf(self, V):

        return (1 + np.tanh((V - self.V_1) / self.V_2)) / 2

    def tau_m(self, V):

        return 1 / np.cosh((V - self.V_3) / (2 * self.V_4))

    def w_inf(self, V):

        return (1 + np.tanh((V - self.V_3) / self.V_4)) / 2

    def tau_w(self, V):

        return 1 / np.cosh((V - self.V_3) / (2 * self.V_4))

    def get_calcium_current(self, V):

        I_Ca = self.g_Ca * self.m_inf(V) * (V - self.V_Ca)
        return I_Ca

    def get_potassium_current(self, V, w):

        I_K = self.g_K * w * (V - self.V_K)
        return I_K

    def get_leak_current(self, V):

        I_L = self.g_L * (V - self.V_L)
        return I_L

    def get_currents(self, V, w):

        I_Ca = self.get_calcium_current(V)
        I_K = self.get_potassium_current(V, w)
        I_L = self.get_leak_current(V)

        return I_Ca, I_K, I_L

    def get_initial_state(self):

        return [self.V0, self.w0]

    def set_initial_state(self, Y):

        self.V0, self.w0 = Y

    def __call__(self, Y, I=0):

        V, w = Y

        I_Ca = self.get_calcium_current(V)
        I_K = self.get_potassium_current(V, w)
        I_L = self.get_leak_current(V)
        dVdt = (I - I_Ca - I_K - I_L) / self.C
        dwdt = self.phi * (self.w_inf(V) - w) / self.tau_w(V)

        return [dVdt, dwdt]